# Exploratory Data Analysis

In [ ]:
from pathlib import Path
from typing import Tuple

import pandas as pd
from scipy.io.arff import loadarff
from scipy.io.arff._arffread import MetaData

In [ ]:
DATA_PATH = Path("../data/file65ef3a759daf.arff")

In [ ]:
def load_data(path_: Path) -> Tuple[pd.DataFrame, MetaData]:
    """Loads the .arff file (incl. metadata) and converts to utf-8.

    Parameters
    -------
    path_ : Path
            Path of the data.

    Returns
    -------
    data : pd.DataFrame
            Data as a dataframe.
    meta : scipy.io.arff._arffread.Metadata
            Metadata of the dataset.
    """
    # load df and metadata from .arff
    data, meta = loadarff(path_)
    data = pd.DataFrame(data)

    # remove b string from data
    str_df = data.select_dtypes([object])
    str_df = str_df.stack().str.decode("utf-8").unstack()
    data = pd.concat([str_df, data.select_dtypes(exclude=[object])], axis=1)

    return data, meta

In [ ]:
data, meta = load_data(DATA_PATH)

In [ ]:
meta

In [ ]:
print("Dimensions of the dataset:", data.shape)

In [ ]:
data.head()

In [ ]:
data["Violation.Type"].value_counts()

In [ ]:
data["Race"].value_counts()

In [ ]:
data["Violation.Type"].value_counts()